# Reference Sites

*   [YORO v3](https://deep-learning-study.tistory.com/411)
*   [Dataset](https://aihub.or.kr/aidata/27727)



# Google Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd /content/gdrive/MyDrive/DeepLearning/Project/HawkEye
!ls -al

# Import Package

In [ ]:
from __future__ import division

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np

# Class

In [ ]:
# Dummy layer for object's (of nn.Module) direct connection on forward function for simple processing
class EmptyLayer(nn.Module) :
    def __init__(self) : super(EmptyLayer, self).__init__()

# Layer save anchors for detection about bounding box
class DetectionLayer(nn.Module) :
    def __init__(self, anchors) :
        super(DetectionLayer, self).__init__()
        self.anchors = anchors

# Functions

*The neural network configuration brought up by the author of [the paper](https://github.com/pjreddie/darknet/blob/master/cfg/yolov3.cfg)*

In [ ]:
# Configuration file read
def parse_cfg(cfgFile) :
    f = open(cfgFile, 'r')
    lines = f.read().split('\n')

    lines = [x for x in lines if len(x) > 0] # empty delete
    lines = [x for x in lines if x[0] != '#'] # comment delete
    lines = [x.rstrip().lstrip() for x in lines] # space delete

    block = {}
    blocks = []

    for line in lines :
        if line[0] == '[' : # start new block
            if len(block) != 0 : # not empty
                blocks.append(block)
                block = {}
            
            block['type'] = line[1 : -1].rstrip()
        else :
            key, value = line.split('=')
            block[key.rstrip()] = value.lstrip()
        
    blocks.append(block)

    return blocks

# Build a Pytorch module for blocks of a configuration file.
def create_modules(blocks) :
    net_info = blocks[0] # save input and pre-processing data
    module_list = nn.ModuleList()

    prev_filters = 3 # RGB(=3) filter
    output_filters = [] # each block's filters count append

    for idx, x in enumerate(blocks[1:]) :
        module = nn.Sequential() # Use Sequential layer

        if x['type'] == 'convolutional' : # block type confirm
            # get layer's infomation
            activation = x["activation"]
            try :
                batch_normalize = int(x['batch_normalize'])
                bias = False
            except :
                batch_normalize = 0
                bias = True

            filters = int(x['filters'])
            padding = int(x['pad'])
            kernel_size = int(x['size'])
            stride = int(x['stride'])

            if padding : pad = (kernel_size - 1) // 2
            else : pad = 0

            # add layer(2D convolutional layer)
            conv = nn.Conv2d(prev_filters, filters, kernel_size, stride, pad, bias=bias)
            module.add_module('conv_{0}'.format(idx), conv)

            # add layer(Batch norm layer)
            if batch_normalize :
                bn = nn.BatchNorm2d(filters)
                module.add_module('batch_norm_{0}'.format(idx), bn)

            # check activation, Leaky ReLU or Linear
            if activation == 'leaky' :
                activn = nn.LeakyReLU(0.1, inplace=True)
                module.add_module('leaky_{0}'.format(idx), activn)

        elif x['type'] == 'upsample' : # block type == upsampling
            stride = int(x['stride'])
            upsample = nn.Upsample(scale_factor=2, mode='bilinear')
            module.add_module('upsample_{}'.format(idx), upsample)

        elif x['type'] == 'route' : # block type == route
            x['layers'] = x['layers'].split(',') # get layer's parameter and split
            start = int(x['layers'][0])

            try : end = int(x['layers'][1])
            except : end = 0

            # positive
            if start > 0 : start = start - idx
            if end > 0 : end = end - idx
            route = EmptyLayer()
            module.add_module('route_{0}'.format(idx), route)
            
            # negative
            if end < 0 : filters = output_filters[idx + start] + output_filters[idx + end]
            else : filters = output_filters[idx + start]

        elif x['type'] == 'shortcut' : # block type == skip connection(shortcut)
            shortcut = EmptyLayer()
            module.add_module('shortcut_{}'.format(idx), shortcut)

        elif x['type'] == 'yolo' : # block type == YOLO
            mask = x['mask'].split(',')
            mask = [int(x) for x in mask]

            anchors = x['anchors'].split(',')
            anchors = [int(a) for a in anchors]
            anchors = [(anchors[i], anchors[i+1]) for i in range(0, len(anchors), 2)]
            anchors = [anchors[i] for i in mask]
            detection = DetectionLayer(anchors)
            module.add_module('Detection_{}'.format(idx), detection)

        module_list.append(module)
        prev_filters = filters
        output_filters.append(filters)

    return (net_info, module_list)

# Main

In [ ]:
blocks = parse_cfg("./yolo_v3.cfg")
print(create_modules(blocks))